In [1]:
!git clone -q https://github.com/soumik12345/point-cloud-segmentation

In [9]:
import sys

sys.path.append("point-cloud-segmentation")

from point_seg import ShapeNetCoreLoaderInMemory, ShapeNetCoreLoader
from point_seg import models

import tensorflow as tf

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
data_loader = ShapeNetCoreLoader(object_category="Airplane")
train_dataset, val_dataset = data_loader.get_datasets(batch_size=32)

1078935552/1078925405 [==============================] - 21s 0us/step


100%|██████████| 4045/4045 [00:00<00:00, 28262.14it/s]



Number of Point Files with keys: 3694
Number of Point Files without keys: 351
Total training files: 2955.
Total validation files: 739.


In [4]:
x, y = next(iter(train_dataset))
print(x.shape, y.shape)

num_points = x.shape[1]
num_classes = y.shape[-1]

(32, 1024, 3) (32, 1024, 5)


In [5]:
baseline_segmentation_model = models.get_baseline_segmentation_model(
    num_points, num_classes
)
baseline_segmentation_model.summary()

Model: "baseline_pointnet_segmentation_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 3)]    0                                            
__________________________________________________________________________________________________
input_transformation_block_1_co (None, 1024, 64)     256         input_1[0][0]                    
__________________________________________________________________________________________________
input_transformation_block_1_ba (None, 1024, 64)     256         input_transformation_block_1_conv
__________________________________________________________________________________________________
input_transformation_block_1_re (None, 1024, 64)     0           input_transformation_block_1_batc
_______________________________________________________________

In [11]:
initial_lr = 0.001
momentum = 0.9
epochs = 60  # Should be 200 as per https://github.com/charlesq34/pointnet/blob/master/part_seg/train.py#L19.
drop_every = 20
decay_factor = 0.5


# Reference: https://www.pyimagesearch.com/2019/07/22/keras-learning-rate-schedules-and-decay/
def lr_schedule(epoch):
    exp = np.floor((1 + epoch) / drop_every)
    new_lr = initial_lr * (decay_factor ** exp)
    return new_lr


lr_callback = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: lr_schedule(epoch), verbose=True
)

In [ ]:
def run_experiment():
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    baseline_segmentation_model = models.get_baseline_segmentation_model(
        num_points, num_classes
    )
    baseline_segmentation_model.compile(
        optimizer=optimizer, loss="categorical_crossentropy"
    )

    logs_dir = f'logs_{datetime.utcnow().strftime("%y%m%d-%H%M%S")}'
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)

    baseline_segmentation_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=epochs,
        callbacks=[tb_callback, lr_callback],
    )


run_experiment()

Epoch 1/60

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
12/93 [==>...........................] - ETA: 2:46 - loss: 4.7005